In [1]:
from typing import Dict
import flwr as fl

from flower_helpers import (create_model, get_weights, test, 
                            load_data, load_stored_tff)
from config import  (NUM_ROUNDS, SERVER_ADDRESS, TRAIN_SIZE, VAL_PORTION, 
                    TEST_SIZE, BATCH_SIZE, LEARNING_RATE, 
                    EPOCHS, FRAC_FIT, FRAC_EVAL, MIN_FIT,
                    MIN_EVAL, MIN_AVAIL, FIT_CONFIG_FN,
                    NUM_CLIENTS, CLIENT_RESOURCES, NON_IID,
                    RAY_ARGS, NUM_CLASSES, TFF_DATA_DIR, 
                    MODEL_NAME, PRE_TRAINED, DOUBLE_TRAIN)
from client import FlowerClient

In [2]:
if NON_IID:
    # non-iid dataset from tff (train and test are already split)
    trainloaders, valloaders, testloader = load_stored_tff(TFF_DATA_DIR, 
                                                           BATCH_SIZE,
                                                           DOUBLE_TRAIN)
else:
    # iid dataset from huggingface
    trainloaders, valloaders, testloader = load_data(MODEL_NAME, TEST_SIZE, 
                                                    TRAIN_SIZE, VAL_PORTION, 
                                                    BATCH_SIZE, NUM_CLIENTS, 
                                                    NUM_CLASSES)

Found cached dataset cifar100 (C:/Users/Jean/.cache/huggingface/datasets/cifar100/cifar100/1.0.0/f365c8b725c23e8f0f8d725c3641234d9331cd2f62919d1381d1baa5b3ba3142)


  0%|          | 0/2 [00:00<?, ?it/s]

Loading cached shuffled indices for dataset at C:\Users\Jean\.cache\huggingface\datasets\cifar100\cifar100\1.0.0\f365c8b725c23e8f0f8d725c3641234d9331cd2f62919d1381d1baa5b3ba3142\cache-fa578d015bc4721c.arrow
Loading cached shuffled indices for dataset at C:\Users\Jean\.cache\huggingface\datasets\cifar100\cifar100\1.0.0\f365c8b725c23e8f0f8d725c3641234d9331cd2f62919d1381d1baa5b3ba3142\cache-ebaa3534d5b9a71d.arrow


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [3]:
net = create_model(MODEL_NAME, NUM_CLASSES, PRE_TRAINED)
init_weights = get_weights(net)
MODEL_CONFIG = net.config
# Convert the weights (np.ndarray) to parameters (bytes)
init_param = fl.common.ndarrays_to_parameters(init_weights)
# del the net as we don't need it anymore
del net

In [4]:
# server side evaluation function
def evaluate(server_round: int, params: fl.common.NDArrays,
             config: Dict[str, fl.common.Scalar]):
    data_size, metrics = test(MODEL_CONFIG, params, testloader)
    # changing the name of the metric to avoid confusion
    metrics['test_loss'] = metrics.pop('loss')
    metrics['test_accuracy'] = metrics.pop('accuracy')
    return metrics['test_loss'], metrics

def weighted_average_eval(metrics):
    weighted_train_loss = 0
    weighted_train_accuracy = 0
    for c in metrics: # c is a tuple (num_examples, metrics) for each client
        weighted_train_loss += c[0] * c[1]['val_loss']
        weighted_train_accuracy += c[0] * c[1]['val_accuracy']
    
    aggregated_metrics = {'val_loss': weighted_train_loss / sum([c[0] for c in metrics]),
            'val_accuracy': weighted_train_accuracy / sum([c[0] for c in metrics])}
    print('\t',aggregated_metrics)
    return aggregated_metrics

def weighted_average_fit(metrics):
    # print(metrics)
    weighted_train_loss = 0
    weighted_train_accuracy = 0
    for c in metrics: # c is a tuple (num_examples, metrics) for each client
        # metrics for each epoch is included, we only need the last one
        weighted_train_loss += c[0] * c[1]['train_loss']
        weighted_train_accuracy += c[0] * c[1]['train_accuracy']
    
    aggregated_metrics = {'train_loss': weighted_train_loss / sum([c[0] for c in metrics]),
            'train_accuracy': weighted_train_accuracy / sum([c[0] for c in metrics])}
    print('\t',aggregated_metrics)
    return aggregated_metrics

In [5]:
strategy = fl.server.strategy.FedAvg(
    fraction_fit=FRAC_FIT,
    fraction_evaluate=FRAC_EVAL,
    min_fit_clients=MIN_FIT,
    min_evaluate_clients=MIN_EVAL,
    min_available_clients=MIN_AVAIL,
    
    fit_metrics_aggregation_fn=weighted_average_fit,
    evaluate_metrics_aggregation_fn=weighted_average_eval,
    evaluate_fn=evaluate,
    on_fit_config_fn=FIT_CONFIG_FN,
    
    initial_parameters=init_param,
)

In [6]:
print('num clients:', NUM_CLIENTS)
print('num rounds:', NUM_ROUNDS)
print('--'*20)
print('client training set size:', [len(t.dataset) for t in trainloaders])
print('client validation set size:', [len(v.dataset) for v in valloaders])
print('test set size:', len(testloader.dataset))
print('--'*20)
print('model name:', MODEL_NAME)
print('num classes:', NUM_CLASSES)
print('pre-trained:', PRE_TRAINED)
print('learning rate:', LEARNING_RATE)
print('batch size:', BATCH_SIZE)
print('epochs:', EPOCHS)

num clients: 5
num rounds: 50
----------------------------------------
client training set size: [180, 180, 180, 180, 180]
client validation set size: [20, 20, 20, 20, 20]
test set size: 100
----------------------------------------
model name: google/vit-base-patch16-224
num classes: 100
pre-trained: True
learning rate: 0.0001
batch size: 32
epochs: 1


In [7]:
fl.server.start_server(server_address=SERVER_ADDRESS, 
                       config=fl.server.ServerConfig(num_rounds=NUM_ROUNDS), 
                       strategy=strategy)

INFO flwr 2023-04-15 17:17:13,987 | app.py:139 | Starting Flower server, config: ServerConfig(num_rounds=50, round_timeout=None)
INFO flwr 2023-04-15 17:17:14,025 | app.py:152 | Flower ECE: gRPC server running (50 rounds), SSL is disabled
INFO flwr 2023-04-15 17:17:14,026 | server.py:86 | Initializing global parameters
INFO flwr 2023-04-15 17:17:14,027 | server.py:266 | Using initial parameters provided by strategy
INFO flwr 2023-04-15 17:17:14,028 | server.py:88 | Evaluating initial parameters
INFO flwr 2023-04-15 17:17:35,400 | server.py:91 | initial parameters (loss, other metrics): 19.52888298034668, {'test_loss': 19.52888298034668, 'test_accuracy': 0.0}
INFO flwr 2023-04-15 17:17:35,401 | server.py:101 | FL starting
DEBUG flwr 2023-04-15 17:17:51,783 | server.py:215 | fit_round 1: strategy sampled 2 clients (out of 2)
DEBUG flwr 2023-04-15 17:21:01,681 | server.py:229 | fit_round 1 received 2 results and 0 failures


	 {'train_loss': 4.369683265686035, 'train_accuracy': 0.025}


INFO flwr 2023-04-15 17:21:11,422 | server.py:116 | fit progress: (1, 18.33440589904785, {'test_loss': 18.33440589904785, 'test_accuracy': 0.0}, 216.02059110000846)
DEBUG flwr 2023-04-15 17:21:11,423 | server.py:165 | evaluate_round 1: strategy sampled 2 clients (out of 2)
DEBUG flwr 2023-04-15 17:21:30,189 | server.py:179 | evaluate_round 1 received 2 results and 0 failures
DEBUG flwr 2023-04-15 17:21:30,189 | server.py:215 | fit_round 2: strategy sampled 2 clients (out of 2)


	 {'val_loss': 4.651551723480225, 'val_accuracy': 0.025}


DEBUG flwr 2023-04-15 17:24:47,738 | server.py:229 | fit_round 2 received 2 results and 0 failures


...

DEBUG flwr 2023-04-15 20:53:21,302 | server.py:229 | fit_round 50 received 2 results and 0 failures


	 {'train_loss': 1.4432869193115039e-05, 'train_accuracy': 1.0}


INFO flwr 2023-04-15 20:53:31,101 | server.py:116 | fit progress: (50, 9.499307632446289, {'test_loss': 9.499307632446289, 'test_accuracy': 0.46}, 12955.700250399997)
DEBUG flwr 2023-04-15 20:53:31,101 | server.py:165 | evaluate_round 50: strategy sampled 2 clients (out of 2)
DEBUG flwr 2023-04-15 20:53:50,362 | server.py:179 | evaluate_round 50 received 2 results and 0 failures
INFO flwr 2023-04-15 20:53:50,362 | server.py:144 | FL finished in 12974.962032300013
INFO flwr 2023-04-15 20:53:50,397 | app.py:202 | app_fit: losses_distributed [(1, 4.651551723480225), (2, 4.481055974960327), (3, 4.362421035766602), (4, 4.2336366176605225), (5, 4.055148363113403), (6, 4.009121775627136), (7, 3.954677700996399), (8, 3.8307955265045166), (9, 3.757102608680725), (10, 3.7171268463134766), (11, 3.6587166786193848), (12, 3.5828269720077515), (13, 3.496229410171509), (14, 3.470556616783142), (15, 3.4126806259155273), (16, 3.375787138938904), (17, 3.3697344064712524), (18, 3.3565053939819336), (19, 

	 {'val_loss': 3.111065626144409, 'val_accuracy': 0.375}


History (loss, distributed):
	round 1: 4.651551723480225
	round 2: 4.481055974960327
	round 3: 4.362421035766602
	round 4: 4.2336366176605225
	round 5: 4.055148363113403
	round 6: 4.009121775627136
	round 7: 3.954677700996399
	round 8: 3.8307955265045166
	round 9: 3.757102608680725
	round 10: 3.7171268463134766
	round 11: 3.6587166786193848
	round 12: 3.5828269720077515
	round 13: 3.496229410171509
	round 14: 3.470556616783142
	round 15: 3.4126806259155273
	round 16: 3.375787138938904
	round 17: 3.3697344064712524
	round 18: 3.3565053939819336
	round 19: 3.3768781423568726
	round 20: 3.306207060813904
	round 21: 3.3139429092407227
	round 22: 3.2789942026138306
	round 23: 3.2633821964263916
	round 24: 3.225769877433777
	round 25: 3.187385082244873
	round 26: 3.1927536725997925
	round 27: 3.191200613975525
	round 28: 3.174075961112976
	round 29: 3.129457116127014
	round 30: 3.0924837589263916
	round 31: 3.097269654273987
	round 32: 3.113898515701294
	round 33: 3.1322388648986816
	round 3